<a href="https://colab.research.google.com/github/jasonsgraham/nlp_notes/blob/main/nlp_starter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup #

In [2]:
# %%sh
# pip install -q --upgrade transformers
# pip install -q --upgrade wandb
# pip install -q --upgrade mlflow

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
allennlp 2.9.3 requires transformers<4.19,>=4.1, but you have transformers 4.20.1 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
allennlp 2.9.3 requires transformers<4.19,>=4.1, but you have transformers 4.20.1 which is incompatible.


Check if notebook is running in Colab or Kaggle

In [32]:
import json
from pathlib import Path

import numpy as np
import pandas as pd
from scipy import sparse
from tqdm import tqdm

pd.options.display.width = 180
pd.options.display.max_colwidth = 120
pd.set_option('max_columns', 500)
pd.set_option('max_rows', 500)
pd.set_option('display.max_rows', 500)

In [26]:
import sys
GOOGLE_COLAB = 'google.colab' in sys.modules

if GOOGLE_COLAB:
  data_dir = Path('/content/drive/MyDrive/Colab Notebooks/input/AI4Code')
  output_dir = Path('/content/drive/MyDrive/Colab Notebooks/output/AI4Code')
  train_parquet_file = data_dir / 'train.parquet'
else:
  data_dir = data_dir = Path('../input/AI4Code')
  output_dir = Path('./')
  train_parquet_file = Path('../input/trainparquet/train.parquet')

# Ordering the Cells #

In [33]:
df_orders = pd.read_csv(
    data_dir / 'train_orders.csv',
    index_col='id',
    squeeze=True,
).str.split()  # Split the string representation of cell_ids into a list

df_orders

id
00001756c60be8    [1862f0a6, 448eb224, 2a9e43d6, 7e2f170a, 038b763d, 77e56113, 2eefe0ef, 1ae087ab, 0beab1cd, 8ffe0b25, 9a78ab76, 0d136...
00015c83e2717b    [2e94bd7a, 3e99dee9, b5e286ea, da4f7550, c417225b, 51e3cd89, 2600b4eb, 75b65993, cf195f8b, 25699d02, 72b3201a, f2c75...
0001bdd4021779    [3fdc37be, 073782ca, 8ea7263c, 80543cd8, 38310c80, 073e27e5, 015d52a4, ad7679ef, 7fde4f04, 07c52510, 0a1a7a39, 0bcd3...
0001daf4c2c76d    [97266564, a898e555, 86605076, 76cc2642, ef279279, df6c939f, 2476da96, 00f87d0a, ae93e8e6, 58aadb1d, d20b0094, 986fd...
0002115f48f982                                 [9ec225f0, 18281c6c, e3b6b115, 4a044c54, 365fe576, a3188e54, b3f6e12d, ee7655ca, 84125b7a]
                                                                           ...                                                           
fffc30d5a0bc46    [09727c0c, ff1ea6a0, ddfef603, a01ce9b3, 3ba953ee, bf92a015, f4a0492a, 095812e6, 53125cfe, aa32a700, 63340e73, 06d8c...
fffc3b44869198    [978a5137, fa

In [27]:
NUM_TRAIN = 10000


def read_notebook(path):
    return (
        pd.read_json(
            path,
            dtype={'cell_type': 'category', 'source': 'str'})
        .assign(id=path.stem)
        .rename_axis('cell_id')
    )

if train_parquet_file.exists():
  df = pd.read_parquet(train_parquet_file)
else:
  paths_train = list((data_dir / 'train').glob('*.json'))[:NUM_TRAIN]
  notebooks_train = [
      read_notebook(path) for path in tqdm(paths_train, desc='Train NBs')
  ]
  df = (
      pd.concat(notebooks_train)
      .set_index('id', append=True)
      .swaplevel()
      .sort_index(level='id', sort_remaining=False)
  )
  #df.to_parquet(train_parquet_file)

In [28]:
df_orders.shape

(139256,)

In [37]:
pd.set_option('display.max_rows', 500)
df

cell_type                                                                                                                   source
id             cell_id                                                                                                                                    
0002115f48f982 18281c6c      code  import numpy as np # linear algebra\nimport pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)\nimport ...
               e3b6b115      code                                                               df = pd.read_csv('../input/metadata_train.csv')\ndf.info()
               4a044c54      code                                                                                                                df.head()
               365fe576      code  #let's check if targets are consistent within the same measurement id\ntargets = df.groupby('id_measurement')[['targ...
               a3188e54      code  sns.countplot(x='target',data=targets)\n# it should be only "1" and "0" but we have cases where target is not consit...
...                           ...                                                                                                                      ...
ffe8d0aa5e7d68 b9f0782a  markdown                                                                                                            ### Modelling
               3492f280  markdown  Now one of the important step in this task, we know that this is a regression task, previously we seen skewness in o...
               eea09e6e  markdown  We will made our regression model, now we have come to end our task, in this section we will capture some parameter ...
               54ffd613  markdown                                                                                                                ### Ridge
               3c645477  markdown                                                                                                         #### Ridge Model

[453294 rows x 2 columns]

In [13]:
nb_id = df.index.unique('id')[6]
# Get the correct order
cell_order = df_orders.loc[nb_id]
nb = df.loc[nb_id, :]
print("The ordered notebook:")
#nb.loc[cell_order, :]
cell_order

The ordered notebook:


['91d97bb2',
 '4e6f32f6',
 '0aeca210',
 'cadfdb16',
 'fe39c117',
 'bbc5f229',
 '91194a54',
 '46cc92d1',
 '9d6ea72b',
 'd0f88604',
 '8df28832',
 'd3345791',
 '8c01934e',
 '18b4e8d6',
 'c012109d',
 '83ff0598',
 'd35f13e5',
 'cfba39db',
 'fbc1c206',
 'ace03b55',
 '671b78af',
 '61d3ad05',
 '6d6683ea',
 '2f41ebe2',
 '9dae7ae9']

In [21]:
def get_ranks(base, derived):
    return [base.index(d) for d in derived]

cell_ranks = get_ranks(cell_order, list(nb.index))
nb.insert(0, 'rank', cell_ranks)

nb

ValueError: '4e6f32f6' is not in list

In [15]:
from pandas.testing import assert_frame_equal

assert_frame_equal(nb.loc[cell_order, :], nb.sort_values('rank'))

In [16]:
df_orders_ = df_orders.to_frame().join(
    df.reset_index('cell_id').groupby('id')['cell_id'].apply(list),
    how='right',
)

ranks = {}
for id_, cell_order, cell_id in df_orders_.itertuples():
    ranks[id_] = {'cell_id': cell_id, 'rank': get_ranks(cell_order, cell_id)}

df_ranks = (
    pd.DataFrame
    .from_dict(ranks, orient='index')
    .rename_axis('id')
    .apply(pd.Series.explode)
    .set_index('cell_id', append=True)
)

df_ranks

rank
id             cell_id      
000c0a9b2fef4d 1087237d    2
               d7209f1f    4
               daf5b8ee    6
               e404213c    7
               2bad59b0    8
...                      ...
fffc63ff750064 56aa8da7   25
               411b85d9    1
               e7e67119    6
               8b54cf58   22
               b3c6bc16   18

[461166 rows x 1 columns]

# Splits #

The `df_ancestors.csv` file identifies groups of notebooks derived from a common origin, that is, notebooks belonging to the same forking tree.

In [ ]:
df_ancestors = pd.read_csv(data_dir / 'train_ancestors.csv', index_col='id')

In [ ]:
df_ancestors

In [ ]:
from sklearn.model_selection import GroupShuffleSplit

NVALID = 0.1  # size of validation set

splitter = GroupShuffleSplit(n_splits=1, test_size=NVALID, random_state=0)

# Split, keeping notebooks with a common origin (ancestor_id) together
ids = df.index.unique('id')
ancestors = df_ancestors.loc[ids, 'ancestor_id']
ids_train, ids_valid = next(splitter.split(ids, groups=ancestors))
ids_train, ids_valid = ids[ids_train], ids[ids_valid]

df_train = df.loc[ids_train, :]
df_valid = df.loc[ids_valid, :]

In [ ]:
df_valid

# Feature Engineering #

Let's generate [tf-idf features](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html#sklearn.feature_extraction.text.TfidfTransformer) to use with our ranking model. These features will help our model learn what kinds of words tend to occur most often at various positions within a notebook.

## AI4Code Extract all functions, variables... names


(see, https://www.kaggle.com/code/haithamaliryan/ai4code-extract-all-functions-variables-names) Upvote if this is useful to you.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Training set
tfidf = TfidfVectorizer(min_df=0.01)
X_train = tfidf.fit_transform(df_train['source'].astype(str))
# Rank of each cell within the notebook
y_train = df_ranks.loc[ids_train].to_numpy()
# Number of cells in each notebook
groups = df_ranks.loc[ids_train].groupby('id').size().to_numpy()

Now let's add the code cell ordering as a feature. We'll append a column that enumerates the code cells in the correct order, like `1, 2, 3, 4, ...`, while having the dummy value `0` for all markdown cells. This feature will help the model learn to put the code cells in the correct order.

In [ ]:
code=nb.loc[nb.cell_type=="code"]

In [ ]:
code

In [ ]:
import tokenize
import io

code.loc['33ff3073','source']

code_text = tokenize.generate_tokens(io.StringIO(code.loc['33ff3073','source']).readline)
[tok for tok in code_text]

In [ ]:
# Extract only function names, variables, comments then we can join them
code_text = tokenize.generate_tokens(io.StringIO(code.loc['33ff3073','source']).readline)
[tok.string for tok in code_text if tok.type==53 or tok.type==55]

In [ ]:
# Add code cell ordering
X_train = sparse.hstack((
    X_train,
    np.where(
        df_train['cell_type'] == 'code',
        df_train.groupby(['id', 'cell_type']).cumcount().to_numpy() + 1,
        0,
    ).reshape(-1, 1)
))

print(X_train.shape)

# Train #

We'll use the ranking algorithm provided by XGBoost.

In [ ]:
from xgboost import XGBRanker

model = XGBRanker(
    min_child_weight=10,
    subsample=0.5,
    tree_method='hist',
)

model.fit(X_train, y_train, group=groups)

# Evaluate #

Now let's see how well our model learned to order Kaggle notebook cells. We'll evaluate predictions on the validation set with a variant of the Kendall tau correlation.

## Validation set ##

First we'll create features for the validation set just like we did for the training set.

In [ ]:
# Validation set
X_valid = tfidf.transform(df_valid['source'].astype(str))
# The metric uses cell ids
y_valid = df_orders.loc[ids_valid]

X_valid = sparse.hstack((
    X_valid,
    np.where(
        df_valid['cell_type'] == 'code',
        df_valid.groupby(['id', 'cell_type']).cumcount().to_numpy() + 1,
        0,
    ).reshape(-1, 1)
))

Here we'll use the model to predict the rank of each cell within its notebook and then convert these ranks into a list of ordered cell ids.

In [ ]:
y_pred = pd.DataFrame({'rank': model.predict(X_valid)}, index=df_valid.index)
y_pred = (
    y_pred
    .sort_values(['id', 'rank'])  # Sort the cells in each notebook by their rank.
                                  # The cell_ids are now in the order the model predicted.
    .reset_index('cell_id')  # Convert the cell_id index into a column.
    .groupby('id')['cell_id'].apply(list)  # Group the cell_ids for each notebook into a list.
)
y_pred.head(10)

Now let's examine a notebook to see how the model did.

In [ ]:
nb_id = df_valid.index.get_level_values('id').unique()[8]

display(df.loc[nb_id])
display(df.loc[nb_id].loc[y_pred.loc[nb_id]])

## Metric ##

This competition uses a variant of the [Kendall tau correlation](https://www.kaggle.com/competitions/AI4Code/overview/evaluation), which will measure how close to the correct order our predicted orderings are. See this notebook for more on this metric: [Competition Metric - Kendall Tau Correlation](https://www.kaggle.com/code/ryanholbrook/competition-metric-kendall-tau-correlation/notebook).

In [ ]:
from bisect import bisect


def count_inversions(a):
    inversions = 0
    sorted_so_far = []
    for i, u in enumerate(a):
        j = bisect(sorted_so_far, u)
        inversions += i - j
        sorted_so_far.insert(j, u)
    return inversions


def kendall_tau(ground_truth, predictions):
    total_inversions = 0
    total_2max = 0  # twice the maximum possible inversions across all instances
    for gt, pred in zip(ground_truth, predictions):
        ranks = [gt.index(x) for x in pred]  # rank predicted order in terms of ground truth
        total_inversions += count_inversions(ranks)
        n = len(gt)
        total_2max += n * (n - 1)
    return 1 - 4 * total_inversions / total_2max

Let's test the metric with a dummy submission created from the ids of the shuffled notebooks.

In [ ]:
y_dummy = df_valid.reset_index('cell_id').groupby('id')['cell_id'].apply(list)
dummy_score = kendall_tau(y_valid, y_dummy)

mlflow.log_metric("Dummy_Score", dummy_score)

Comparing this to the score on the predictions, we can see that our model was indeed able to improve the cell ordering somewhat.

In [ ]:
prediction_score = kendall_tau(y_valid, y_pred)
mlflow.log_metric("Prediction_Score", prediction_score)

# Submission #

To create a submission for this competition, we'll apply our model to the notebooks in the test set. Note that this is a **Code Competition**, which means that the test data we see here is only a small sample. When we submit our notebook for scoring, this example data will be replaced with the full test set of about 20,000 notebooks.

First we load the data.

In [ ]:
paths_test = list((data_dir / 'test').glob('*.json'))
notebooks_test = [
    read_notebook(path) for path in tqdm(paths_test, desc='Test NBs')
]
df_test = (
    pd.concat(notebooks_test)
    .set_index('id', append=True)
    .swaplevel()
    .sort_index(level='id', sort_remaining=False)
)

Then create the tf-idf and code cell features.

In [ ]:
X_test = tfidf.transform(df_test['source'].astype(str))
X_test = sparse.hstack((
    X_test,
    np.where(
        df_test['cell_type'] == 'code',
        df_test.groupby(['id', 'cell_type']).cumcount().to_numpy() + 1,
        0,
    ).reshape(-1, 1)
))

And then create predictions on the test set.

In [ ]:
y_infer = pd.DataFrame({'rank': model.predict(X_test)}, index=df_test.index)
y_infer = y_infer.sort_values(['id', 'rank']).reset_index('cell_id').groupby('id')['cell_id'].apply(list)
y_infer

The `sample_submission.csv` file shows what a correctly formatted submission must look like. We'll just use it as a visual check, but you might like to directly modify the values of sample submission instead. (This would help prevent failed submissions due to missing notebook ids or incorrectly named columns, for instance.)